# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [6]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Analyzer Templates

Below is a collection of analyzer templates designed to extract fields from various input file types.

These templates are highly customizable, allowing you to modify them to suit your specific needs. For additional verified templates from Microsoft, please visit [here](../analyzer_templates/README.md).

In [7]:
extraction_templates = {
    "invoice":            ('../analyzer_templates/invoice.json',         '../data/invoice.pdf'            ),
    "chart":              ('../analyzer_templates/image_chart.json',     '../data/pieChart.jpg'           ),
    "call_recording":     ('../analyzer_templates/call_recording_analytics.json', '../data/callCenterRecording.mp3'),
    "conversation_audio": ('../analyzer_templates/conversational_audio_analytics.json', '../data/callCenterRecording.mp3'),
    "marketing_video":    ('../analyzer_templates/marketing_video.json', '../data/video.mp4'              ),
    "PostCallAnalyticsOnText":  ('../CU-Demo-Assets/analyzers/PostCallAnalyticsOnText.json', '../CU-Demo-Assets/en-us-conversation-transcripts/sample0 - Insurance - Customer calls about purchasing life insurance.wav.results.txt'),
    "PostCallAnalyticsCustomized": ('../CU-Demo-Assets/analyzers/PostCallAnalyticsCustomized.json', '../CU-Demo-Assets/en-US-mono-audio/sample4 - Healthcare - Customer calls about new policy quote.wav'),
    "PostCallAnalyticsCustomizedDE": ('../CU-Demo-Assets/analyzers/PostCallAnalyticsWithGermanResult.json', '../CU-Demo-Assets/de-DE-mono-audio/sample4 - Healthcare - Customer calls about new policy quote- German.wav'),
    "ImageTextAnalysis": ('../CU-Demo-Assets/analyzers/imageTextAnalysis.json', '../CU-Demo-Assets/en-us-images-w-text/Designer.png'),
    "MultiLanguageDemo": ('../CU-Demo-Assets/analyzers/MultiLanguageDemo.json', '../CU-Demo-Assets/pt-BR-mono-audio/sample4 - Healthcare - Customer calls about new policy quote - Brazilian.wav'),
    "MultiLanguageDemoOnText":  ('../CU-Demo-Assets/analyzers/MultiLanguageDemoOnText.json', '../CU-Demo-Assets/pt-BR-conversation-transcripts/sample6 - Financial - Customer calls about transferring funds - Brazilian.wav.results.txt'),
}

Specify the analyzer template you want to use and provide a name for the analyzer to be created based on the template.

In [8]:
import uuid

ANALYZER_TEMPLATE = "PostCallAnalyticsOnText"
ANALYZER_ID = "cu-demo-" + str(uuid.uuid4())

(analyzer_template_path, analyzer_sample_file_path) = extraction_templates[ANALYZER_TEMPLATE]

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [9]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Linux-6.5.0-1025-azure-x86_64-with-glibc2.36)'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 400
Response headers:
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'IMDS/150.870.65.1475'
    'x-ms-request-id': 'cf2bc6ab-63c7-4f5e-a240-d0e516a52b3e'
    'Date': 'Thu, 13 Feb 2025 19:56:35 GMT'
    'Content-Length': '88'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Re

## List all analyzers created in your resource

After the analyzer is successfully created, we can use it to analyze our input files.

In [10]:
response = client.get_all_analyzers()
print(f"Number of analyzers in your resource: {len(response['value'])}")
print(f"First 3 analyzer details: {json.dumps(response['value'], indent=2)}")

Number of analyzers in your resource: 12
First 3 analyzer details: [
  {
    "analyzerId": "prebuilt-read",
    "description": "Extract content elements such as words, barcodes, and formulas from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": false,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "prebuilt-layout",
    "description": "Extract various content and layout elements such as words, paragraphs, and tables from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": true,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "ImageTextAnalysis",
    "description": "Example to extract text from images and summarizes it.",
    "tags": {
      "pr

## Create Analyzer from the Template

In [11]:
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzer cu-demo-7a7a10a8-299e-4bb5-8a34-3e68cda2440b create request accepted.
INFO:python.content_understanding_client:Request bc3acf4d-a79b-46ee-961c-d59bb4bc2225 in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.30 seconds.


{
  "id": "bc3acf4d-a79b-46ee-961c-d59bb4bc2225",
  "status": "Succeeded",
  "result": {
    "analyzerId": "cu-demo-7a7a10a8-299e-4bb5-8a34-3e68cda2440b",
    "description": "Conversation analytics chat and pre-trancsribed audio transcripts",
    "tags": {
      "projectId": "",
      "templateId": "text-2024-12-01"
    },
    "createdAt": "2025-02-13T19:56:36Z",
    "lastModifiedAt": "2025-02-13T19:56:37Z",
    "config": {
      "returnDetails": false,
      "disableContentFiltering": false
    },
    "fieldSchema": {
      "name": "CallCenterConversationAnalysis",
      "fields": {
        "Summary": {
          "type": "string",
          "method": "generate",
          "description": "A one-paragraph summary"
        },
        "Topics": {
          "type": "array",
          "method": "generate",
          "description": "Top 5 topics mentioned",
          "items": {
            "type": "string"
          }
        },
        "Companies": {
          "type": "array",
          "me

# Extract Conversation (WEBVTT) from either Batch Transcription JSON or Content Understanding JSON

## Extract Fields Using the Analyzer

After the analyzer is successfully created, we can use it to analyze our input files.

In [12]:
response = client.begin_analyze(ANALYZER_ID, file_location=analyzer_sample_file_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzing file ../CU-Demo-Assets/en-us-conversation-transcripts/sample0 - Insurance - Customer calls about purchasing life insurance.wav.results.txt with analyzer: cu-demo-7a7a10a8-299e-4bb5-8a34-3e68cda2440b
INFO:python.content_understanding_client:Request e715680d-a33c-4180-9609-c868feaf6a85 in progress ...
INFO:python.content_understanding_client:Request e715680d-a33c-4180-9609-c868feaf6a85 in progress ...
INFO:python.content_understanding_client:Request result is ready after 4.30 seconds.


{
  "id": "e715680d-a33c-4180-9609-c868feaf6a85",
  "status": "Succeeded",
  "result": {
    "analyzerId": "cu-demo-7a7a10a8-299e-4bb5-8a34-3e68cda2440b",
    "apiVersion": "2024-12-01-preview",
    "createdAt": "2025-02-13T19:56:41Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "```\n```WEBVTT\\n\\n00:00.080 --> 00:02.240\\n<v Agent>Hi there, thank you for calling Contoso.\\n\\n00:02.240 --> 00:03.200\\n<v Agent>My name is Sam.\\n\\n00:03.440 --> 00:04.720\\n<v Agent>How can I help you today?\\n\\n00:05.520 --> 00:07.680\\n<v Customer>Good afternoon, my name is Mary Adams.\\n\\n00:08.560 --> 00:10.720\\n<v Customer>I want to buy life insurance for my husband.\\n\\n00:11.520 --> 00:13.520\\n<v Customer>Can you please tell me how much it costs?\\n\\n00:14.400 --> 00:17.200\\n<v Agent>I'm going to help you right now with your life insurance needs.\\n\\n00:18.080 --> 00:19.760\\n<v Agent>Are you the owner of the policy?\\n\\n00:20.560 --> 00:28.960\\n<v Customer>My h

## Clean Up
Optionally, delete the sample analyzer from your resource. In typical usage scenarios, you would analyze multiple files using the same analyzer.

In [13]:
client.delete_analyzer(ANALYZER_ID)

INFO:python.content_understanding_client:Analyzer cu-demo-7a7a10a8-299e-4bb5-8a34-3e68cda2440b deleted.


<Response [204]>